# Создание нейронной сети для распознавания растений и деревьев на изображениях.

## Датасет
описать датасет. откуда он, сколько картинок в целом, в каждом классе, их размеры и тд

## Нейронная сеть

- Выбираем архитектуру
    - Сверточный модуль:
        - Сверточный слой, выделяющий 32 признака по фильтрам размерами 3 и глубиной 3 (RGB каналы изображений для разделения цветов) с функцией активации `ReLU`
        - Слой макс-пуллинга, сжимающий карты первичных признаков в 2 раза
        - Сверточный слой, выделяющий 64 признака по фильтрам размерами 3 и глубиной 3 с активацией `ReLU`
        - Слой макс-пуллинга, сжимающий карты вторичных признаков в 2 раза
        - Преобразование карт признаков в единый массив размерами 64
    - Полносвязный модуль:
        - Скрытый полносвязный слой из 64 нейронов с функцией активации ReLU
        - Выходной полносвязный слой из 35 нейронов (количество классов) с функцией активации `softmax`

- Импортируем библиотеку `tensorflow` для создания нейронной сети

In [9]:
import tensorflow as tf

- Составляем модель нейронной сети по описанной выше архитектуре:

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(64, 64, 3)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(15, activation='softmax')
])

- Компилируем модель с оптимизатором `Adam` и рассчетой потерь по `кросс-энтропии`:

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Создадим класс `Dataset` для удобной работы с датасетом и тренировке нашей нейросети:
- Импортируем `PIL` для открытия изображений, `numpy` для преобразования изображений в массивы чисел и `os` для работы с файлами:

In [12]:
from PIL import Image
import numpy as np
import os

- Загрузим тренировочные и проверочные картинки из папки 'dataset':

In [13]:
train_images = []
train_labels = []

src = 'dataset/training'
src_classes = os.listdir(src)

for i in range(len(src_classes)):
    class_path = os.path.join(src, src_classes[i])
    class_images = os.listdir(class_path)
    for j in range(len(class_images)):
        image_path = os.path.join(class_path, class_images[j])
        image = Image.open(image_path).convert('RGB').resize((64, 64))

        image_array = np.array(image)
        image_label = np.zeros(len(src_classes))
        image_label[i] = 1

        train_images.append(image_array)
        train_labels.append(image_label)

test_images = []
test_labels = []

src = 'dataset/validation'
src_classes = os.listdir(src)

for i in range(len(src_classes)):
    class_path = os.path.join(src, src_classes[i])
    class_images = os.listdir(class_path)
    for j in range(len(class_images)):
        image_path = os.path.join(class_path, class_images[j])
        image = Image.open(image_path).convert('RGB').resize((64, 64))

        image_array = np.array(image)
        image_label = np.zeros(len(src_classes))
        image_label[i] = 1

        test_images.append(image_array)
        test_labels.append(image_label)

Обучаем нашу нейросеть:

In [31]:
model.fit(np.array(train_images), np.array(train_labels), epochs=100, 
          validation_data=(np.array(test_images), np.array(test_labels)))

Epoch 1/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 1.3622e-05 - val_accuracy: 0.5248 - val_loss: 0.7193
Epoch 2/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 1.3357e-05 - val_accuracy: 0.5248 - val_loss: 0.7212
Epoch 3/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 1.3388e-05 - val_accuracy: 0.5248 - val_loss: 0.7222
Epoch 4/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 1.3630e-05 - val_accuracy: 0.5248 - val_loss: 0.7242
Epoch 5/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 1.1376e-05 - val_accuracy: 0.5248 - val_loss: 0.7250
Epoch 6/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 1.1960e-05 - val_accuracy: 0.5248 - val_loss: 0.7284
Epoch 7/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 1.1437e-05 - val_accuracy: 0.5248 - val_loss: 0.7286
Epoch 8/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - los

Делаем несколько тестовых предсказаний:

In [22]:
test1 = np.array(Image.open('dataset/test/bilimbi/bilimbi88.jpg').resize((64, 64)))
preds = model.predict(np.expand_dims(test1, axis=0))
print(src_classes[np.argmax(preds)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
bilimbi


In [26]:
test1 = np.array(Image.open('dataset/test/guava/guava78.jpg').resize((64, 64)))
preds = model.predict(np.expand_dims(test1, axis=0))
print(src_classes[np.argmax(preds)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
guava


In [27]:
test1 = np.array(Image.open('dataset/test/melon/melon556.jpg').resize((64, 64)))
preds = model.predict(np.expand_dims(test1, axis=0))
print(src_classes[np.argmax(preds)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
melon


In [32]:
test1 = np.array(Image.open('dataset/test/pomelo/pomelo45.jpg').resize((64, 64)))
preds = model.predict(np.expand_dims(test1, axis=0))
print(src_classes[np.argmax(preds)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
pomelo
